In [1]:
import pyspark

In [ ]:
# my path for py4j lib 
#import sys
#sys.path.append("/opt/spark-3.4.4/python")
#sys.path.append("/opt/spark-3.4.4/python/lib/py4j-0.10.9.7-src.zip")


In [ ]:
#pyspark.__file__

In [ ]:
#print(pyspark.__version__)

In [ ]:
#!python --version


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/29 12:49:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# main data source : https://divvy-tripdata.s3.amazonaws.com/index.html 

In [ ]:
#!wget https://divvy-tripdata.s3.amazonaws.com/202004-divvy-tripdata.zip

In [4]:
import zipfile
import pandas as pd

In [5]:
# Define the path to the zip file
zip_file_path = '202004-divvy-tripdata.zip.1'

In [6]:
# Open the zip file and list its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # List all file names inside the zip file
    zip_ref.printdir()

    # Get the first file name in the zip (assumed to be the CSV)
    csv_file = zip_ref.namelist()[0]

    # Read the CSV file into a pandas DataFrame directly from the zip without extracting
    with zip_ref.open(csv_file) as file:
        df = pd.read_csv(file)

# Print the first few rows of the DataFrame
df.head()

File Name                                             Modified             Size
202004-divvy-tripdata.csv                      2020-06-01 10:43:50     14254024
__MACOSX/                                      2020-06-01 10:49:02            0
__MACOSX/._202004-divvy-tripdata.csv           2020-06-01 10:43:50          434


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A847FADBBC638E45,docked_bike,2020-04-26 17:45:14,2020-04-26 18:12:03,Eckhart Park,86,Lincoln Ave & Diversey Pkwy,152.0,41.8964,-87.6610,41.9322,-87.6586,member
1,5405B80E996FF60D,docked_bike,2020-04-17 17:08:54,2020-04-17 17:17:03,Drake Ave & Fullerton Ave,503,Kosciuszko Park,499.0,41.9244,-87.7154,41.9306,-87.7238,member
2,5DD24A79A4E006F4,docked_bike,2020-04-01 17:54:13,2020-04-01 18:08:36,McClurg Ct & Erie St,142,Indiana Ave & Roosevelt Rd,255.0,41.8945,-87.6179,41.8679,-87.6230,member
3,2A59BBDF5CDBA725,docked_bike,2020-04-07 12:50:19,2020-04-07 13:02:31,California Ave & Division St,216,Wood St & Augusta Blvd,657.0,41.9030,-87.6975,41.8992,-87.6722,member
4,27AD306C119C6158,docked_bike,2020-04-18 10:22:59,2020-04-18 11:15:54,Rush St & Hubbard St,125,Sheridan Rd & Lawrence Ave,323.0,41.8902,-87.6262,41.9695,-87.6547,casual


In [7]:
import io

In [8]:
# Define a temporary file path
temp_csv_path = 'temp_bikes.csv'

# Open the zip file and extract the CSV file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # List all file names inside the zip file
    zip_ref.printdir()

    # Get the first file name in the zip (assumed to be the CSV)
    csv_file = zip_ref.namelist()[0]

    # Extract the CSV file and save it temporarily
    with zip_ref.open(csv_file) as file:
        with open(temp_csv_path, 'wb') as temp_file:
            temp_file.write(file.read())

# Now read the CSV file with Spark
df = spark.read.option("header", "true").csv(temp_csv_path)

# Show the first few rows of the DataFrame
df.show(5)


File Name                                             Modified             Size
202004-divvy-tripdata.csv                      2020-06-01 10:43:50     14254024
__MACOSX/                                      2020-06-01 10:49:02            0
__MACOSX/._202004-divvy-tripdata.csv           2020-06-01 10:43:50          434
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------+--------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat|start_lng|end_lat| end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------+--------+-------------+
|A847FADBBC638E45|  docked_bike|2020-04-26 17:45:14|2020-04-26 18:12:03|        Eckhar

In [9]:
df.schema

StructType([StructField('ride_id', StringType(), True), StructField('rideable_type', StringType(), True), StructField('started_at', StringType(), True), StructField('ended_at', StringType(), True), StructField('start_station_name', StringType(), True), StructField('start_station_id', StringType(), True), StructField('end_station_name', StringType(), True), StructField('end_station_id', StringType(), True), StructField('start_lat', StringType(), True), StructField('start_lng', StringType(), True), StructField('end_lat', StringType(), True), StructField('end_lng', StringType(), True), StructField('member_casual', StringType(), True)])

In [10]:
df.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- ended_at: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: string (nullable = true)
 |-- start_lng: string (nullable = true)
 |-- end_lat: string (nullable = true)
 |-- end_lng: string (nullable = true)
 |-- member_casual: string (nullable = true)



In [11]:
from pyspark.sql.functions import col, to_timestamp

In [12]:
# Convert 'started_at' and 'ended_at' columns to timestamp format
df = df.withColumn('started_at', to_timestamp('started_at', 'yyyy-MM-dd HH:mm:ss')) \
       .withColumn('ended_at', to_timestamp('ended_at', 'yyyy-MM-dd HH:mm:ss'))

# Show the updated DataFrame schema and the first few rows
df.printSchema()



root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: string (nullable = true)
 |-- start_lng: string (nullable = true)
 |-- end_lat: string (nullable = true)
 |-- end_lng: string (nullable = true)
 |-- member_casual: string (nullable = true)



In [13]:
df.show(2)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------+--------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat|start_lng|end_lat| end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+---------+-------+--------+-------------+
|A847FADBBC638E45|  docked_bike|2020-04-26 17:45:14|2020-04-26 18:12:03|        Eckhart Park|              86|Lincoln Ave & Div...|           152|  41.8964|  -87.661|41.9322|-87.6586|       member|
|5405B80E996FF60D|  docked_bike|2020-04-17 17:08:54|2020-04-17 17:17:03|Drake Ave & Fulle...|             503|     Kosciuszko Park|           499|  41.9244| -87.7154|41.9306|-87.7238|       member|
+---------

In [14]:
import pyarrow as pa
import pyarrow.parquet as pq

In [15]:
df = df.withColumn("started_at", to_timestamp(df["started_at"]))
df = df.withColumn("ended_at", to_timestamp(df["ended_at"]))

In [16]:
df = df.withColumn("start_lat", col("start_lat").cast("float"))
df = df.withColumn("start_lng", col("start_lng").cast("float"))
df = df.withColumn("end_lat", col("end_lat").cast("float"))
df = df.withColumn("end_lng", col("end_lng").cast("float"))


In [23]:
df.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: float (nullable = true)
 |-- start_lng: float (nullable = true)
 |-- end_lat: float (nullable = true)
 |-- end_lng: float (nullable = true)
 |-- member_casual: string (nullable = true)



In [24]:
df.write.parquet('ride_data_test_play.parquet')